In [1]:
import pandas as pd
import plotly.express as px

# Import required functions for Dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [2]:
# Import pickle data
df = pd.read_pickle('dashboard_data.pkl')
df

,group,sector,sector_percentage
0,Other,Agriculture,27.198127
1,Other,Arts,1.821642
2,Other,Clothing,5.022102
3,Other,Construction,0.964454
4,Other,Education,4.674505
5,Other,Entertainment,0.101011
6,Other,Food,20.968425
7,Other,Health,1.424790
8,Other,Housing,4.706090
9,Other,Manufacturing,0.963985


In [3]:
# Create app object

my_app = JupyterDash(__name__)

# Layout definition

my_app.layout = html.Div([
    html.H1('Percentage distribution of Sector Loans per Country Group',
           style={"textAlign": "center"}),                                # Centers title 
    html.Font('Country Group Select',
             style={'font-weight': 'bold'}),                              # Bolds title for dropdown box
    dcc.Dropdown(options=df.loc[:,'group'].unique(),                      # Available options for dropdown box
                 value=df.loc[:,'group'].unique(),                        # Selected options for dropdown box on first load
                 multi=True,                                              # Allows multiple options to be selected at once
                 style={"width":250},                                     # Defines width of the dropdown box
                 id='dropdown_country'
                ),
    html.Br(),
    html.Font('Sector Select',
             style={'font-weight': 'bold'}),                              # Bolds title for dropdown box
    dcc.Dropdown(options=df.loc[:,'sector'].unique(),                     # Available options for dropdown box
                 value=df.loc[:,'sector'].unique(),                       # Selected options for dropdown box on first load
                 multi=True,                                              # Allows multiple options to be selected at once
                 id='dropdown_sector'
                ),
    dcc.Graph(
              id='graph'
             )
])        

# Callback definition
@my_app.callback(
    Output('graph', 'figure'),
    Input('dropdown_country', 'value'),
    Input('dropdown_sector', 'value')
)
def update_text_size(group, sector):
    
    df_select = df.loc[(df['group'].isin(group)) & (df['sector'].isin(sector))] # Selects data based on dropdown filters

    df_heatmap_pivot = df_select.pivot(index='group', columns='sector')['sector_percentage'] # Required formatting for heatmap

    plot_heatmap = px.imshow(
        df_heatmap_pivot, 
        x=df_heatmap_pivot.columns,                                 # Passes the columns of the df as values for x-axis
        y=df_heatmap_pivot.index,                                   # Passes the indexs of the df as values for x-axis
        title='<b>Distribution of Loans Per Country Group and Sector</b>',
    )

    plot_heatmap.update_layout(
        title_x=0.5,                                                # Horizontal alignment of the title
    )

    plot_heatmap.update_layout(
        coloraxis_colorbar={'title':"% of Loans within Group"},     # Changes colourbar text
        xaxis=dict(title='Sector'),                                 # Customises y-axis title
        yaxis=dict(title='Country Group'),                          # Customises x-axis title
    )

    plot_heatmap.update_xaxes(tickangle=90)                         # Text rotation, otherwise text overlaps with colorbar

    return plot_heatmap
    
# Run app

if __name__ == '__main__':
    my_app.run_server(mode='inline')